In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypothyroidism"
cohort = "GSE75685"

# Input paths
in_trait_dir = "../../input/GEO/Hypothyroidism"
in_cohort_dir = "../../input/GEO/Hypothyroidism/GSE75685"

# Output paths
out_data_file = "../../output/preprocess/Hypothyroidism/GSE75685.csv"
out_gene_data_file = "../../output/preprocess/Hypothyroidism/gene_data/GSE75685.csv"
out_clinical_data_file = "../../output/preprocess/Hypothyroidism/clinical_data/GSE75685.csv"
json_path = "../../output/preprocess/Hypothyroidism/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Mexican Patients with Breast Cancer"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: Tumor Sample of Breast Cancer'], 1: ['gender: Female'], 2: ['rna ng/ul: 1083', 'rna ng/ul: 343', 'rna ng/ul: 111', 'rna ng/ul: 307', 'rna ng/ul: 401', 'rna ng/ul: 475', 'rna ng/ul: 728', 'rna ng/ul: 143.6', 'rna ng/ul: 224.7', 'rna ng/ul: 1458.3', 'rna ng/ul: 164', 'rna ng/ul: 370.2', 'rna ng/ul: 419.5', 'rna ng/ul: 693.6', 'rna ng/ul: 291.4', 'rna ng/ul: 1566.4', 'rna ng/ul: 69', 'rna ng/ul: 625.4', 'rna ng/ul: 151.6', 'rna ng/ul: 127.7', 'rna ng/ul: 1116.8', 'rna ng/ul: 333.9', 'rna ng/ul: 182.1', 'rna ng/ul: 437.4', 'rna ng/ul: 439', 'rna ng/ul: 178.2', 'rna ng/ul: 1365', 'rna ng/ul: 670', 'rna ng/ul: 840.6', 'rna ng/ul: 725'], 3: ['rna 260/280 (nanodrop): 2.15', 'rna 260/280 (nanodrop): 2.16', 'rna 260/280 (nanodrop): 2.19', 'rna 2

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at background and sample characteristics, this appears to be a breast cancer dataset
# There's no explicit mention of gene expression, but as a SuperSeries from GEO, it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Hypothyroidism), examining row 21 which contains personal pathological history
# There is a value "personal pathological hystory: Hypothyroidism" in row 21
trait_row = 21

# For age, examining row 19 which contains "age at diagnosis" values
age_row = 19

# For gender, examining row 1 which contains "gender: Female"
# All samples are female, which means gender is a constant feature (not useful for association studies)
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (1 for Hypothyroidism, 0 for others)"""
    if value is None or pd.isna(value):
        return None
    # Extract the part after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Check if "Hypothyroidism" is mentioned in the value
    if "Hypothyroidism" in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if value is None or pd.isna(value):
        return None
    # Extract the part after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Try to convert to integer
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

# Not needed since gender data is constant, but for completeness:
def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None or pd.isna(value):
        return None
    # Extract the part after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    # Convert to binary
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we should extract clinical features
if trait_row is not None and 'clinical_data' in locals():
    # Use the helper function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=None  # No need for gender conversion since it's constant
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM1963528': [0.0, 45.0], 'GSM1963529': [0.0, 41.0], 'GSM1963530': [0.0, 59.0], 'GSM1963531': [0.0, 57.0], 'GSM1963532': [0.0, 42.0], 'GSM1963533': [0.0, 49.0], 'GSM1963534': [0.0, 59.0], 'GSM1963535': [0.0, 54.0], 'GSM1963536': [0.0, 54.0], 'GSM1963537': [0.0, 31.0], 'GSM1963538': [0.0, 70.0], 'GSM1963539': [0.0, 44.0], 'GSM1963540': [0.0, 50.0], 'GSM1963541': [0.0, 42.0], 'GSM1963542': [0.0, 56.0], 'GSM1963543': [1.0, 51.0], 'GSM1963544': [0.0, 58.0], 'GSM1963545': [0.0, 55.0], 'GSM1963546': [0.0, 71.0], 'GSM1963547': [0.0, 42.0], 'GSM1963548': [0.0, 41.0], 'GSM1963549': [0.0, 40.0], 'GSM1963550': [0.0, 57.0], 'GSM1963551': [0.0, 62.0], 'GSM1963552': [0.0, 87.0], 'GSM1963553': [0.0, 36.0], 'GSM1963554': [0.0, 50.0], 'GSM1963555': [0.0, 45.0], 'GSM1963556': [0.0, 43.0], 'GSM1963557': [0.0, 42.0], 'GSM1963558': [0.0, 43.0], 'GSM1963559': [0.0, 44.0], 'GSM1963560': [0.0, 43.0], 'GSM1963561': [0.0, 48.0], 'GSM1963562': [0.0, 45.0], 'GSM1963563': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 45220 rows
First 20 gene IDs:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown in the previous step (1, 2, 3, etc.) are numeric identifiers
# and not standard human gene symbols like BRCA1, TP53, etc.
# These numeric identifiers need to be mapped to proper gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 3353129 rows

Gene annotation preview (first few rows):
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': ['170', '168', '166', '164', '162'], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify mapping columns
# From previous output, we can see the gene expression data uses 'ID' as identifiers
# The annotation data has 'ID' and 'GENE_SYMBOL' columns that we need for mapping

print("Identifying mapping columns...")
# Check if the relevant columns exist in gene_annotation
if 'ID' in gene_annotation.columns and 'GENE_SYMBOL' in gene_annotation.columns:
    print("Found ID and GENE_SYMBOL columns for mapping")
    
    # 2. Get gene mapping dataframe
    print("Creating gene mapping dataframe...")
    # Extract ID and GENE_SYMBOL columns for mapping
    mapping_df = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
    print(f"Created mapping dataframe with {len(mapping_df)} rows")
    print("First few rows of mapping dataframe:")
    print(preview_df(mapping_df))
    
    # 3. Apply gene mapping to convert probe-level data to gene expression data
    print("\nConverting probe-level measurements to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Converted to gene expression data with {len(gene_data)} genes")
    print("First few gene symbols:")
    print(gene_data.index[:10].tolist())
    
    # Check if we have any mapped genes
    if len(gene_data) == 0:
        print("Warning: No genes were mapped. Check if the annotation data is correct.")
        is_gene_available = False
    else:
        # Normalize gene symbols to ensure consistency
        print("\nNormalizing gene symbols...")
        gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(gene_data)} genes")
        print("First few normalized gene symbols:")
        print(gene_data.index[:10].tolist())
        
        # Save the gene expression data
        print("\nSaving gene expression data...")
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene data saved to {out_gene_data_file}")
        
else:
    print("Error: Required columns for mapping not found in annotation data")
    is_gene_available = False


Identifying mapping columns...
Found ID and GENE_SYMBOL columns for mapping
Creating gene mapping dataframe...
Created mapping dataframe with 35657 rows
First few rows of mapping dataframe:
{'ID': ['12', '14', '15', '16', '18'], 'Gene': ['APOBEC3B', 'ATP11B', 'LOC100132006', 'DNAJA1', 'EHMT2']}

Converting probe-level measurements to gene expression data...


Converted to gene expression data with 18380 genes
First few gene symbols:
['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS']

Normalizing gene symbols...
After normalization: 17901 genes
First few normalized gene symbols:
['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']

Saving gene expression data...


Gene data saved to ../../output/preprocess/Hypothyroidism/gene_data/GSE75685.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (17901, 54)
Gene data shape after normalization: (17901, 54)


Normalized gene data saved to ../../output/preprocess/Hypothyroidism/gene_data/GSE75685.csv
Clinical data saved to ../../output/preprocess/Hypothyroidism/clinical_data/GSE75685.csv
Linked data shape: (54, 17903)

Handling missing values...


After missing value handling, linked data shape: (54, 17903)

Evaluating feature bias...
For the feature 'Hypothyroidism', the least common label is '1.0' with 1 occurrences. This represents 1.85% of the dataset.
The distribution of the feature 'Hypothyroidism' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 43.0
  50% (Median): 49.5
  75%: 56.75
Min: 31.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

Trait bias evaluation result: True

Dataset usability: False
Dataset is not usable for Hypothyroidism association studies. Data not saved.
